In [8]:
import pandas as pd
import pickle

# Load parquet file into pandas dataframe
df_latent_inf = pd.read_parquet(
    "/Users/sidbaskaran/Desktop/research/HyperDAS/axbench/axbench/concept10/prod_2b_l10_v1/inference/latent_eval_data.parquet"
)
df_generate_train_data = pd.read_parquet(
    "/Users/sidbaskaran/Desktop/research/HyperDAS/axbench/axbench/concept10/prod_2b_l10_v1/generate/train_data.parquet"
)

# Load pickle file containing inference state
with open(
    "/Users/sidbaskaran/Desktop/research/HyperDAS/axbench/axbench/concept10/prod_2b_l10_v1/inference/latent_inference_state.pkl",
    "rb",
) as f:
    inference_state = pickle.load(f)

In [24]:
import pandas as pd

df = pd.read_parquet("../../assets/data/axbench/inference/steering_data.parquet")

# Print column names before renaming
print("Original columns:", df.columns.tolist())

# Check if the columns exist before renaming
df = df.rename(
    columns={
        "HyperReFT_steered_perplexity": "HyperReFT_perplexity",
        "PromptSteering_steered_perplexity": "PromptSteering_perplexity",
    }
)
df.to_parquet("../../assets/data/axbench/inference/steering_data.parquet")

Original columns: ['dataset_name', 'concept_id', 'input_concept', 'input_id', 'factor', 'original_prompt', 'steered_input', 'input', 'sae_link', 'sae_id', 'HyperReFT_steered_generation', 'HyperReFT_steered_perplexity', 'PromptSteering_steered_generation', 'PromptSteering_steered_perplexity']


In [25]:
df = pd.read_parquet("../../assets/data/axbench/inference/steering_data.parquet")

In [26]:
df

,dataset_name,concept_id,input_concept,input_id,factor,original_prompt,steered_input,input,sae_link,sae_id,HyperReFT_steered_generation,HyperReFT_perplexity,PromptSteering_steered_generation,PromptSteering_perplexity
0,AlpacaEval,0,references to specific dates and publication i...,0,0.0,What are different drawers I should have for c...,<start_of_turn>user\nWhen responding to questi...,<start_of_turn>user\nWhat are different drawer...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,14874,\n**Different drawers for clothes:**\n\n**1. F...,4.327478,1. **Main drawer**\n - To store out-of-season ...,4.896508
1,AlpacaEval,0,references to specific dates and publication i...,0,0.0,What are different drawers I should have for c...,<start_of_turn>user\nWhen responding to questi...,<start_of_turn>user\nWhat are different drawer...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,14874,\n**Different drawers for clothes:**\n\n**1. F...,4.327478,1. **Main drawer**\n - To store out-of-season ...,4.896508
2,AlpacaEval,0,references to specific dates and publication i...,1,0.0,how do I appear in a video call as if I'd be a...,<start_of_turn>user\nWhen responding to questi...,<start_of_turn>user\nhow do I appear in a vide...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,14874,1. **Use a sleep mask and earplugs:** A sleep ...,6.987170,1. **Use a recording device or screen recordin...,6.189692
3,AlpacaEval,0,references to specific dates and publication i...,1,0.0,how do I appear in a video call as if I'd be a...,<start_of_turn>user\nWhen responding to questi...,<start_of_turn>user\nhow do I appear in a vide...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,14874,1. **Use a sleep mask and earplugs:** A sleep ...,6.987170,1. **Use a recording device or screen recordin...,6.189692
4,AlpacaEval,0,references to specific dates and publication i...,2,0.0,"In this math problem, we are asked to figure o...",<start_of_turn>user\nWhen responding to questi...,"<start_of_turn>user\nIn this math problem, we ...",https://www.neuronpedia.org/gemma-2-2b/10-gemm...,14874,\n\nThe total profit percentage can be calcula...,38.960632,\n**Total profit percentage = (90% + 92% + 81%...,62.846935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,AlpacaEval,9,patterns or mentions related to numerical data...,5,0.0,Make a list of the top places in the U.S. to v...,<start_of_turn>user\nPlease provide responses ...,<start_of_turn>user\nMake a list of the top pl...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,11674,\n**Top places to visit in the U.S. in Novembe...,4.082453,\n**Top places to visit in the United States i...,4.956636
106,AlpacaEval,9,patterns or mentions related to numerical data...,6,0.0,what is the recommended pattern for creating g...,<start_of_turn>user\nPlease provide responses ...,<start_of_turn>user\nwhat is the recommended p...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,11674,\n**Recommended Pattern for Game Creation in G...,3.727482,\n**Recommended Pattern for Game Development i...,3.621057
107,AlpacaEval,9,patterns or mentions related to numerical data...,7,0.0,GIVE ME AN EXECUTIVE SUMMARY OF LOTTADIGITAL.COM,<start_of_turn>user\nPlease provide responses ...,<start_of_turn>user\nGIVE ME AN EXECUTIVE SUMM...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,11674,\n**Lottadigital.com** is a leading provider o...,4.884633,\n**Lottadigital: An Executive Summary**\n\n**...,5.392300
108,AlpacaEval,9,patterns or mentions related to numerical data...,8,0.0,Write a social media post about the call for c...,<start_of_turn>user\nPlease provide responses ...,<start_of_turn>user\nWrite a social media post...,https://www.neuronpedia.org/gemma-2-2b/10-gemm...,11674,📢 Calling all creative minds and problem-solve...,4.716204,📢 Calling all innovators and problem solvers! ...,5.842714


In [11]:
import json

with open(
    "/Users/sidbaskaran/Desktop/research/HyperDAS/assets/data/gemma-2-2b_10-gemmascope-res-16k.json",
    "r",
) as f:
    data = json.load(f)

In [12]:
data

[{'modelId': 'gemma-2-2b',
  'layer': '10-gemmascope-res-16k',
  'index': '10529',
  'description': 'HTML attributes and their values',
  'explanationModelName': 'gpt-4o-mini',
  'typeName': 'oai_token-act-pair'},
 {'modelId': 'gemma-2-2b',
  'layer': '10-gemmascope-res-16k',
  'index': '10755',
  'description': 'HTML table and element tags',
  'explanationModelName': 'gpt-4o-mini',
  'typeName': 'oai_token-act-pair'},
 {'modelId': 'gemma-2-2b',
  'layer': '10-gemmascope-res-16k',
  'index': '10984',
  'description': 'structured sections of code',
  'explanationModelName': 'gpt-4o-mini',
  'typeName': 'oai_token-act-pair'},
 {'modelId': 'gemma-2-2b',
  'layer': '10-gemmascope-res-16k',
  'index': '11564',
  'description': ' HTML header tags',
  'explanationModelName': 'gpt-4o-mini',
  'typeName': 'oai_token-act-pair'},
 {'modelId': 'gemma-2-2b',
  'layer': '10-gemmascope-res-16k',
  'index': '12088',
  'description': 'product announcements and releases',
  'explanationModelName': 'gpt-

In [7]:
inference_state

{'concept_id': 9}

In [9]:
df_generate_train_data

,input,output,output_concept,concept_genre,category,dataset_category,concept_id
0,Why do some people select private schools over...,There are many reasons why some people choose ...,EEEEE,text,negative,instruction,-1
1,What is a derivative in finance?,"In finance, a derivative is a financial instru...",EEEEE,text,negative,instruction,-1
2,What is underwriting?,Underwriting is the process of evaluating and ...,EEEEE,text,negative,instruction,-1
3,"Total 15 PMs served under Queen Elizabeth II, ...",The last Prime Minister to serve under Queen E...,EEEEE,text,negative,instruction,-1
4,Which NFL teams have gone to the Super Bowl on...,Here are the NFL teams that have gone to the S...,EEEEE,text,negative,instruction,-1
...,...,...,...,...,...,...,...
931,How would you create a class in Python to stor...,"class Car:\n def __init__(self, make: str, ...",patterns or mentions related to numerical data...,code,positive,instruction,9
932,Write a python script to generates random numb...,import random\n\ndef generate_divisible_by_thr...,patterns or mentions related to numerical data...,code,positive,instruction,9
933,Write a Python program to query a MySQL Server...,```python\nimport mysql.connector\n\ndef fetch...,patterns or mentions related to numerical data...,code,positive,instruction,9
934,Create a machine learning classifier in Python...,import numpy as np\nfrom sklearn.model_selecti...,patterns or mentions related to numerical data...,code,positive,instruction,9
